# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=True, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35557,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 1.86 GiB
Comm: tcp://127.0.0.1:36827,Total threads: 4
Dashboard: http://127.0.0.1:36423/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:41445,


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,1050,Quinn,0.276945,0.024964
1,1020,Zelda,-0.142738,-0.224680
2,984,Ingrid,0.524142,-0.990503
3,1011,George,-0.789623,0.843858
4,1015,Bob,-0.646320,-0.275406


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,9.999919e+02,-3.030603e-04,4.099838e-04
std,3.162604e+01,5.773291e-01,5.771544e-01
min,8.510000e+02,-9.999985e-01,-9.999999e-01
25%,9.790000e+02,-4.950484e-01,-4.920303e-01
50%,1.000000e+03,6.382523e-03,6.332303e-03
75%,1.022000e+03,5.041664e-01,5.066032e-01
max,1.156000e+03,9.999998e-01,9.999999e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
927,Patricia,-0.5799321020390915
991,Bob,-0.5627945294469936
1030,Michael,-0.2787098408028858
1014,Xavier,0.5224241920076442
952,Zelda,0.0956709367845294
948,Charlie,0.1821484813284877
1043,Xavier,0.4636088263816165
1066,Laura,0.6154876704114485
979,Ingrid,0.572853898358813


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,1050,Quinn,0.276945,0.024964
1,1020,Zelda,-0.142738,-0.224680
2,984,Ingrid,0.524142,-0.990503
3,1011,George,-0.789623,0.843858
4,1015,Bob,-0.646320,-0.275406


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
df.describe().compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,1050,Quinn,0.276945,0.024964
1,1020,Zelda,-0.142738,-0.224680
2,984,Ingrid,0.524142,-0.990503
3,1011,George,-0.789623,0.843858
4,1015,Bob,-0.646320,-0.275406


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,927,Patricia,-0.579932,NaN
1,991,Bob,-0.562795,NaN
2,1030,Michael,-0.278710,NaN
3,1014,Xavier,0.522424,NaN
4,952,Zelda,0.095671,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-e3c4ce75-ddb5-41ae-85ef-3d3031c2fa0d')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,1050,Quinn,0.276945,0.024964
1,1020,Zelda,-0.142738,-0.224680
2,984,Ingrid,0.524142,-0.990503
3,1011,George,-0.789623,0.843858
4,1015,Bob,-0.646320,-0.275406
...,...,...,...,...
86395,1028,Kevin,-0.770701,0.895359
86396,972,Sarah,-0.856212,-0.416453
86397,1049,Frank,-0.765742,0.953485
86398,1045,Charlie,-0.150348,0.510716


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-27.csv',
 'data/2000-01-15.csv',
 'data/2000-01-10.csv',
 'data/2000-01-01.csv',
 'data/2000-01-13.csv',
 'data/2000-01-14.csv',
 'data/2000-01-21.csv',
 'data/2000-01-30.csv',
 'data/2000-01-29.csv',
 'data/2000-01-28.csv',
 'data/2000-01-20.csv',
 'data/2000-01-11.csv',
 'data/2000-01-07.csv',
 'data/2000-01-22.csv',
 'data/2000-01-08.csv',
 'data/2000-01-02.csv',
 'data/2000-01-04.csv',
 'data/2000-01-05.csv',
 'data/2000-01-17.csv',
 'data/2000-01-23.csv',
 'data/2000-01-26.csv',
 'data/2000-01-16.csv',
 'data/2000-01-24.csv',
 'data/2000-01-03.csv',
 'data/2000-01-06.csv',
 'data/2000-01-18.csv',
 'data/2000-01-12.csv',
 'data/2000-01-19.csv',
 'data/2000-01-25.csv',
 'data/2000-01-09.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-2f3574a1-0871-40ce-8cae-b48f65997e39'),
 Delayed('read_data-fe17d4e3-0043-42a6-bf64-7a01bb115cf3'),
 Delayed('read_data-00d70791-ae26-4fb9-aaa0-011c4bbeff83'),
 Delayed('read_data-29eef97f-5f00-43fb-a2ce-193e035720b2'),
 Delayed('read_data-0b9b6508-3c89-43c4-bf0c-e0ae212fb907'),
 Delayed('read_data-58673a10-721b-4660-8408-1dafec9000b8'),
 Delayed('read_data-fcbb5d4e-e4c1-4255-b395-089e918bcc11'),
 Delayed('read_data-86d650e0-e695-49b8-b89e-4163df76c7d5'),
 Delayed('read_data-c2464978-aed1-450e-8b6f-512bf172d0d8'),
 Delayed('read_data-c72ad9f9-5c55-4efb-a402-b12d7d9c9466'),
 Delayed('read_data-73d61e58-2832-43bf-bbb9-32b3008ffa90'),
 Delayed('read_data-50903442-8a07-4141-916a-8684150b8183'),
 Delayed('read_data-a38315b5-cc71-4833-9cf3-aa221f689e70'),
 Delayed('read_data-2dd78bcb-dafe-4876-896d-2927d3465b12'),
 Delayed('read_data-1c08445f-251b-44dc-a22d-9cac763ee7df'),
 Delayed('read_data-bca66676-812f-4cf0-b19a-ed957a662df2'),
 Delayed('read_data-a02e4d77-0612-4fa7-a

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,1057,Wendy,-0.944425,-0.989789
1,1018,George,0.392865,0.541282
2,1013,Alice,0.807034,-0.358043
3,995,Tim,0.136581,-0.382246
4,994,George,-0.387621,-0.296867


In [26]:
df.describe().compute()

,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,9.999913e+02,-2.463209e-04,4.568442e-04
std,3.163076e+01,5.773315e-01,5.772118e-01
min,8.510000e+02,-9.999985e-01,-9.999999e-01
25%,9.790000e+02,-4.950484e-01,-4.920303e-01
50%,1.000000e+03,6.382523e-03,6.332303e-03
75%,1.022000e+03,5.041664e-01,5.066032e-01
max,1.156000e+03,9.999998e-01,9.999999e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.